## Advanced Login

By default, the client will create a redirect server and launch the user's default browser to show the login page. Neither of these are however required. This is useful for deployed applications that need to be initialized from the command line (also known as 'headless authentication').

Start by initializing the client as usual:

In [1]:
from saxo_apy import SaxoOpenAPIClient
import json

with open("app_config.json", "r") as f:
    config = json.load(f)

client = SaxoOpenAPIClient(config)

Next, use the `.login()` method with the below arguments to disable the browser and redirect server.

You will manually have to open the login page at the provided link, and copy the URL location you land on into the terminal after you enter your credentials.

Note: because no server is listening for the redirect, the browser will show a "This site can't be reached" or "Unable to connect" error. This is however not a problem, as the required data (authorization code) is available in the URL. Copy this value into your command line when prompted to connect the client.

In [ ]:
client.login(launch_browser=False, catch_redirect=False, start_refresh_thread=True)

The command line will look something like this:

```
🌐 navigate to the following web page to log in: https://sim.logonvalidation.net/authorize?.....
📎 paste redirect location (url): < REDIRECT LOCATION GOES HERE >
✅ authorization succeeded - connected to 🛠 SIM environment with 🔧 write / 📈 trade permissions (session ID ea1df9dfd8464915947ba529bd2f02ac)
```

The client is now connected in the same way as using the browser/redirect server:

In [5]:
client.logged_in

True

In [6]:
# while this notebook is running, a background thread is making sure the refresh token gets periodically updated
import threading

print([thread for thread in threading.enumerate() if thread.name == "RefreshThread"])

[<Timer(RefreshThread, started 6266646528)>]


## Authenticating a Deployed Application

Let's assume you have a cloud server running at `my-dev-box.my-domain.net`. We want to deploy a OpenAPI client on this server and authenticate it with Saxo SSO.

We can use the above procedure to authenticate the client from the command line (through SSH for example).

However, we can also use the client to seamlessly authenticate while still using a browser and callback server. We first need to ensure we add a new redirect URL to the application on the Developer Portal:

`http://my-dev-box.my-domain.net:12321/redirect`

Saxo SSO will redirect the user to this location when login credentials are validated. Make sure your cloud instance firewall allows for inbound connections on the chosen port (in this case `12321`), otherwise your server will never be able to catch the callback.

Next, we initialize the client as usual:

In [ ]:
from saxo_apy import SaxoOpenAPIClient
import json

with open("app_config.json", "r") as f:
    config = json.load(f)

client = SaxoOpenAPIClient(config)

The client provides a convenience function to show the available redirect URLs:

In [ ]:
[url.host for url in client.available_redirect_urls]

Next, we select the **second** URL for our login flow:

In [ ]:
client.login(redirect_url=client.available_redirect_urls[1])

Enter your credentials and confirm. Saxo SSO now forwards you to the cloud instance, which should be able to catch the callback and authorize your client.